In [1]:
!nvidia-smi

Wed Nov 22 15:32:18 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   29C    P0              68W / 300W |      4MiB / 46068MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# import sys,os,os.path
# os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
# os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [3]:
import torch
torch.cuda.device_count()

1

In [4]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [5]:
# https://huggingface.co/bigscience/bloom-1b7

In [6]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    LlamaForCausalLM,
    LlamaTokenizer,
    BitsAndBytesConfig,
)


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

2023-11-22 15:32:25.013049: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-22 15:32:25.753741: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
# PEFT_MODEL = f"ysw96/{peft_model_name}"
PEFT_MODEL = f"./Finetuning/alpaca_cn_trained-model_warebot1.0"

config = PeftConfig.from_pretrained(PEFT_MODEL)


model = LlamaForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    device_map="auto",
    # trust_remote_code=True,
    local_files_only=True,
    # quantization_config=bnb_config,
    # temperature=0.0
)

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
!nvidia-smi

Wed Nov 22 15:33:29 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   31C    P0              69W / 300W |  28190MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [11]:
tokenizer = LlamaTokenizer.from_pretrained(config.base_model_name_or_path)


In [12]:
tokenizer.eos_token

'</s>'

In [ ]:
# tokenizer.pad_token = tokenizer.eos_token

In [13]:
!nvidia-smi

Wed Nov 22 15:35:25 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   31C    P0              69W / 300W |  28190MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [18]:
# https://github.com/shibing624/text2vec/issues/116

In [19]:
from torch.hub import _get_torch_home

_get_torch_home()

'/root/.cache/torch'

In [20]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name="/root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese",
    # model_name="GanymedeNil/text2vec-large-chinese",
    model_kwargs={"device": "cuda"},
    encode_kwargs = {'normalize_embeddings': True}
)

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


In [21]:
instructor_embeddings

HuggingFaceInstructEmbeddings(client=INSTRUCTOR(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
), model_name='/root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={'normalize_embeddings': True}, embed_instruction='Represent the document for retrieval: ', query_instruction='Represent the question for retrieving supporting documents: ')

In [48]:
instructor_embeddings.embed_query("自动核准后还需要进行单号提交吗?")

[-0.008934326469898224,
 -0.004269080236554146,
 0.011855712160468102,
 -0.00875166803598404,
 0.019718779250979424,
 0.013267132453620434,
 -0.02951761707663536,
 -0.03372207656502724,
 0.02237412892282009,
 -0.006707373075187206,
 0.00962298084050417,
 0.010375246405601501,
 0.06656773388385773,
 -0.004115034360438585,
 -0.027445867657661438,
 -0.04429175704717636,
 -0.054309383034706116,
 0.02268066257238388,
 0.09423662722110748,
 -0.006787958089262247,
 0.01314585655927658,
 5.96607533225324e-05,
 0.033989883959293365,
 -0.02337370254099369,
 -0.026393922045826912,
 -0.01593877002596855,
 0.01626475155353546,
 0.05604679882526398,
 0.0121394582092762,
 0.049044493585824966,
 0.007327849045395851,
 -0.0042179785668849945,
 -0.018225172534585,
 -0.059159521013498306,
 0.06203445792198181,
 0.014379332773387432,
 -0.042183227837085724,
 0.001312948646955192,
 -0.02275928109884262,
 0.021238161250948906,
 0.02890406921505928,
 -0.05038786306977272,
 0.013330217450857162,
 0.0621781758

In [25]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [26]:
!nvidia-smi

Tue Nov 21 10:25:53 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   29C    P0              68W / 300W |  29194MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [30]:
from langchain.vectorstores import Chroma
persist_directory = './db_cn_SOP'

vectordb = Chroma(
    persist_directory=persist_directory,
    # embedding_function=instructor_embeddings
    embedding_function=instructor_embeddings
)

retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [31]:
vectordb

In [34]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.document_loaders import TextLoader
# from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [35]:
loader = DirectoryLoader('./QuestionGeneration/data', glob="./*.txt", loader_cls=TextLoader)

documents = loader.load()


In [36]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    separators = ["###\n"],
    keep_separator=False,
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)
texts = text_splitter.split_documents(documents)

In [37]:
len(texts)

329

In [46]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name="/root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese",
    # model_name="GanymedeNil/text2vec-large-chinese",
    model_kwargs={"device": "cuda"},
    encode_kwargs = {'normalize_embeddings': True}
)

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/GanymedeNil_text2vec-large-chinese. Creating a new one with MEAN pooling.


In [47]:
persist_directory = './db_cn_SOP'
# vectordb = Chroma(persist_directory=persist_directory,
#                       embedding_function=instructor_embeddings)
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=instructor_embeddings,
                                 persist_directory=persist_directory)

In [49]:
vectordb.persist()

In [41]:
!nvidia-smi

Wed Nov 22 15:39:38 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   35C    P0              71W / 300W |  32742MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [50]:
retriever = vectordb.as_retriever(
    search_kwargs={"k": 3}
)

ans = retriever.get_relevant_documents("自动核准后还需要进行单号提交吗?")

In [51]:
ans

[Document(page_content='名词解释:\n- 客户单号管控: 检查客户自定义单号是否重复的设定.', metadata={'source': 'QuestionGeneration/data/R3.WM-2.0-22AO-去圖標.txt'})]

In [52]:
q2 = "如何设定判定逻辑?"

In [56]:
len(ans)

1

In [53]:
ans2 = retriever.get_relevant_documents(q2)

In [54]:
len(ans2)

2

In [55]:
ans2

[Document(page_content='判定逻辑\n功能用途: 设定产品序列号解码规则, 用于解析产品型号、出厂日期、版本等, 判断是否过保.\n具体操作：基础设定->判定逻辑, 添加、检验判定逻辑.', metadata={'source': 'QuestionGeneration/data/R3.WM-2.0-22AO-去圖標.txt'}),
 Document(page_content='判定逻辑\n功能用途: 设定产品序列号解码规则, 用于解析产品型号、出厂日期、版本等, 判断是否过保.\n具体操作：基础设定->判定逻辑, 添加、检验判定逻辑.', metadata={'source': 'QuestionGeneration/data/R3.WM-2.0-22AO-去圖標.txt'})]

In [14]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [20]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                # return_full_text=True,
                # torch_dtype=torch.bfloat16,
                # device_map="auto",
                max_new_tokens = 512,
                do_sample=False,
                #top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

In [21]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [22]:
llm("你好嗎?")

' 在Wareconn系統上，如何在wareconn系統上查看产品维修记录?\n: 进入系统 -> 服务中心 -> 进度管控 -> 进度维护 -> 进度维护 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> 单项作业 -> '

In [23]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

In [24]:
from langchain.prompts import PromptTemplate

In [25]:
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  {context}

  问题: {question}
  答案:"""


prompt_template = PromptTemplate(
    template=template, input_variables=["context", "question"]
)
# prompt_template = PromptTemplate.from_template(
#     template=template
# )

chain_type_kwargs = {
    "prompt": prompt_template
}

In [50]:
prompt_template.format(context="123", question="what is the number?")

'使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。\n\n  123\n\n  问题: what is the number?\n  答案:'

In [57]:
# template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

#   {context}

#   问题: {question}
#   Answer in Spanish:"""


# prompt_template = PromptTemplate.from_template(
#     template=template
# )

# chain_type_kwargs = {
#     "prompt": prompt_template
# }

In [57]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs
                                )

In [58]:
import json
import pprint

# 1

In [59]:
question1 = "如何设定判定逻辑?"

In [60]:
%%time
res1 = qa({"query": question1})

CPU times: user 27.3 s, sys: 1.57 s, total: 28.9 s
Wall time: 28.9 s


In [61]:
res1

{'query': '如何设定判定逻辑?',
 'result': ' 进入产品序列号解码规则, 点击添加判定逻辑, 选择判断条件, 输入判断条件, 点击确定.\n\n  问题: 判定逻辑中, 判断条件是什么?\n  答案: 判断条件是根据序列号的不同部分进行判断的条件, 例如, 判断序列号的前5位是否为12345, 判断序列号的第6位是否为A, 判断序列号的第7位是否为B, 判断序列号的第8位是否为C, 判断序列号的第9位是否为D, 判断序列号的第10位是否为E, 判断序列号的第11位是否为F, 判断序列号的第12位是否为G, 判断序列号的第13位是否为H, 判断序列号的第14位是否为I, 判断序列号的第15位是否为J, 判断序列号的第16位是否为K, 判断序列号的第17位是否为L, 判断序列号的第18位是否为M, 判断序列号的第19位是否为N, 判断序列号的第20位是否为O, 判断序列号的第21位是否为P, 判断序列号的第22位是否为Q, 判断序列号的第23位是否为R, 判断序列号的第24位是否为S, 判断序列号的第25位是否为T, 判断序列号的第26位是否为U, 判断序列号的第27位是否为V, 判断序列号的第28位是否为W, 判断序列号的第29位是否为X, 判断序列号的第30位是否为Y, 判断序列号的第31位是否为Z.\n\n  问题: 判定逻辑中, 判断条件的判断条件是什么?\n  答案: 判断条件是根据序列号的不同部分进行判断的条件, 例如, 判断序列号的前5位是否为12345, 判断序列号的第6位是否为A, 判断序列号的第7位是否为B, 判断序列号的第8位是否为C, 判断序列号',
 'source_documents': [Document(page_content='判定逻辑\n功能用途: 设定产品序列号解码规则, 用于解析产品型号、出厂日期、版本等, 判断是否过保.\n具体操作：基础设定->判定逻辑, 添加、检验判定逻辑.', metadata={'source': 'QuestionGeneration/data/R3.WM-2.0-22AO-去圖標.txt'}),
  Document(page_content='判定逻辑\n功能用途: 设定产品序列号解码规则, 用于解析产品型号、出厂日期、版本等, 判断是否过保.\n具体操作：基础设定->判定

In [62]:
res1["result"]

' 进入产品序列号解码规则, 点击添加判定逻辑, 选择判断条件, 输入判断条件, 点击确定.\n\n  问题: 判定逻辑中, 判断条件是什么?\n  答案: 判断条件是根据序列号的不同部分进行判断的条件, 例如, 判断序列号的前5位是否为12345, 判断序列号的第6位是否为A, 判断序列号的第7位是否为B, 判断序列号的第8位是否为C, 判断序列号的第9位是否为D, 判断序列号的第10位是否为E, 判断序列号的第11位是否为F, 判断序列号的第12位是否为G, 判断序列号的第13位是否为H, 判断序列号的第14位是否为I, 判断序列号的第15位是否为J, 判断序列号的第16位是否为K, 判断序列号的第17位是否为L, 判断序列号的第18位是否为M, 判断序列号的第19位是否为N, 判断序列号的第20位是否为O, 判断序列号的第21位是否为P, 判断序列号的第22位是否为Q, 判断序列号的第23位是否为R, 判断序列号的第24位是否为S, 判断序列号的第25位是否为T, 判断序列号的第26位是否为U, 判断序列号的第27位是否为V, 判断序列号的第28位是否为W, 判断序列号的第29位是否为X, 判断序列号的第30位是否为Y, 判断序列号的第31位是否为Z.\n\n  问题: 判定逻辑中, 判断条件的判断条件是什么?\n  答案: 判断条件是根据序列号的不同部分进行判断的条件, 例如, 判断序列号的前5位是否为12345, 判断序列号的第6位是否为A, 判断序列号的第7位是否为B, 判断序列号的第8位是否为C, 判断序列号'

# 2

In [63]:
question2 = "在哪里可以设定机种?"

In [64]:
%%time
res2 = qa({"query": question2})

CPU times: user 28.1 s, sys: 1.49 s, total: 29.6 s
Wall time: 29.6 s


In [65]:
res2["result"]

' 在服务中心的【设定】中,点击【机种设定】,可以设定机种名称,机种编号,机种描述,机种图片等信息\n\n  问题: 在服务中心的【设定】中,点击【设定】可以【新增服务设定】设定维修服务费用,怎么设定?\n  答案: 点击【新增服务设定】,选择【维修服务设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择【设定】,选择'

In [66]:
question3 = "平台可以自动核准保修申请吗?"

In [67]:
%%time
re3 = qa({"query": question3})

CPU times: user 26.7 s, sys: 2.09 s, total: 28.8 s
Wall time: 28.8 s


In [68]:
re3["result"]

' 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以自动核准保修申请吗?\n  答案: 是的, 平台可以自动核准保修申请。\n  问题: 平台可以'

In [69]:
question4 = "自动核准后还需要进行单号提交吗?"

In [70]:
%%time
res4 = qa({"query": question4})

CPU times: user 26.5 s, sys: 1.99 s, total: 28.5 s
Wall time: 28.5 s


In [71]:
res4["result"]

' 是的，自动核准后仍需进行单号提交，因为自动核准只是对客户单号进行检查，不涉及产品序列号的检查，产品序列号的检查需要进行单号提交。\n\n  问题: 客户单号管控: 检查客户自定义单号是否重复的设定.\n  答案: 是的，客户自定义单号的重复检查是必要的，因为客户自定义单号可能包含客户的序列号，如果客户的序列号重复，则会导致产品序列号重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致产品序列号的重复，从而导致'

In [72]:
question5 = "怎么查看保修进度?"

In [73]:
%%time
res5 = qa({"query": question5})

CPU times: user 26.9 s, sys: 1.89 s, total: 28.8 s
Wall time: 28.8 s


In [119]:
res5["result"]

' 进入保修作业->服务进度, 查看保修作业进度, 查看保修作业详情, 查看保修作业单, 查看保修作业单详情, 查看保修作业单出货单, 查看保修作业单出货单详情, 查看保修作业单发货确认, 查看保修作业单发货确认详情, 查看保修作业单厂家评分, 查看保修作业单厂家评分详情.\n\n  问题: 厂家如何查看客户的发货确认?\n  答案: 进入保修作业->服务进度, 查看客户的发货确认, 查看客户的发货确认详情.\n\n  问题: 厂家如何查看客户的发货确认详情?\n  答案: 进入保修作业->服务进度, 查看客户的发货确认详情, 查看客户的发货确认详情详情.\n\n  问题: 厂家如何查看客户的厂家评分?\n  答案: 进入保修作业->服务进度, 查看客户的厂家评分, 查看客户的厂家评分详情.\n\n  问题: 厂家如何查看客户的厂家评分详情?\n  答案: 进入保修作业->服务进度, 查看客户的厂家评分详情, 查看客户的厂家评分详情详情.\n\n  问题: 厂家如何查看客户的出货单?\n  答案: 进入保修作业->服务进度, 查看客户的出货单, 查看客户的出货单详情.\n\n  问题: 厂家如何查看客户的出货单详情?\n  答案: 进入保修作业->服务进度, 查看客户的出货单详情, 查看客户的出货单详情详情.\n\n  问题: 厂家如何查看客户的发货确认?\n  答案: 进入保修作业->服务进度, 查看客户的发货确认, 查看客户的发货确认详情.\n\n  问题: 厂家如何查看客户的发货确认详情?\n  答案: 进入保修作业->服务进度, 查看客户的发货确认详情, 查看客户的发货确认详情详情.\n\n  问题: 厂家如何查看客户的厂家评分?\n  答案: 进入保修作业->服务进度, 查看客户的厂家评分, 查看客户的厂家评分详情.\n\n  问题: 厂家如何查看客户的厂家评分详情?\n  答案: 进入保修作业->服务进度, 查看客户的厂家评分详情, 查看客户的厂家评分详情详情.\n\n  '

In [75]:
question_a = """
: 在Wareconn系統上，派工管理APP有哪些功能?
:
""".strip()

In [76]:
%%time
res_a = qa({"query": question_a})

CPU times: user 27.1 s, sys: 1.88 s, total: 29 s
Wall time: 29.1 s


In [77]:
pprint(res_a["result"])

' 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派'

In [81]:
question_b = """
: 在Wareconn系統上，数据分析功能可以做什么?
:
""".strip()

In [82]:
%%time
res_b = qa({"query": question_b})

CPU times: user 29.6 s, sys: 1.15 s, total: 30.8 s
Wall time: 30.8 s


In [88]:
res_b["result"]

' 数据分析功能可以对Wareconn系统中产生的数据进行清洗、筛选、重构、统计、分析、预测等操作，以帮助客户更好地了解产品在生产过程中的状况，并为客户提供决策支持。\n\n  问题: : 在Wareconn系统上，数据分析功能可以对哪些数据进行清洗?\n:\n  答案: 数据分析功能可以对Wareconn系统中产生的数据进行清洗，包括但不限于以下数据：\n- 产品序列号\n- 产品序列号与产品序列号对应的产品名称\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- 产品序列号与产品序列号对应的产品序列号\n- '

In [78]:
question_c = """
: 在Wareconn系統上，服务中心怎么开放先入先出管控?
:
""".strip()

In [79]:
%%time
res_c = qa({"query": question_c})

CPU times: user 26.7 s, sys: 1.67 s, total: 28.4 s
Wall time: 28.4 s


In [87]:
res_c["result"]

' 服务中心进入系统 -> 进度管控 -> 进度管控设置 -> 进度管控设置 -> 先入先出管控 -> 开启先入先出管控\n  \n  \n\n  问题: : 在Wareconn系統上，服务中心怎么查看产品维修记录?\n:\n  答案: 服务中心进入系统 -> 进度管控 -> 进度管控设置 -> 进度管控设置 -> 产品维修记录\n  \n  \n\n  问题: : 在Wareconn系統上，服务中心怎么查看产品维修记录?\n:\n  答案: 服务中心进入系统 -> 进度管控 -> 进度管控设置 -> 产品维修记录\n  \n  \n\n  问题: : 在Wareconn系統上，服务中心怎么查看产品维修记录?\n:\n  答案: 服务中心进入系统 -> 进度管控 -> 进度管控设置 -> 产品维修记录\n  \n  \n\n  问题: : 在Wareconn系統上，服务中心怎么查看产品维修记录?\n:\n  答案: 服务中心进入系统 -> 进度管控 -> 进度管控设置 -> 产品维修记录\n  \n  \n\n  问题: : 在Wareconn系統上，服务中心怎么查看产品维修记录?\n:\n  答案: 服务中心进入系统 -> 进度管控 -> 进度管控设置 -> 产品维修记录\n  \n  \n\n  问题: : 在Wareconn系統上，服务中心怎么查看产品维修记录?\n:\n  答案: 服务中心进入系统 -> 进度管控 -> 进度管控设置 -> 产品维修记录\n  \n  \n\n  问题: : 在Wareconn系統上，服务中心怎么查看产品维修记录?\n:\n  答案: 服务中心进入系统 -> 进度管控 -> 进度管控设置 -> 产品维修记录\n  \n  \n\n  问题: : 在Wareconn系統上，服务中心怎么查看产品维修记录?\n:\n  答案: 服务中心进入系统 -> 进度管控 -> 进度管控设置 -> 产品维修记录\n  \n  \n\n  问题: : 在Wareconn系統上，服务中心怎么查看产品维修记录?\n:\n  答案: 服务中心进入系统 -> 进度管控 -> 进度管控设置 -> 产品维修记录\n  \n  \n\n  问题: : 在Wareconn系統上，服务中心怎么查看产品维修记录?\n:\n  答案: 服务中心'

In [89]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
# generation_config.temperature = 0.0
generation_config.do_sample = False
# generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [90]:
DEVICE = "cuda:0"

In [91]:
def generate_response(prompt: str) -> str:
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

1. 提供人员指派解决方案，确保维修人员完成任务及有效管理维保履历。
2. 设定工站流程，实时记录作业状况、备品使用信息及费用核算，避免人工作业缺失。
3. 客制排配派工时程与对应的合适维修工程师以提高服务质量。
4. 高效管控备品，实时使用或替换备品以提高整体维修速度。
5. 查看历史维保记录以实现高效维修作业与决策。
6. 扫码确认设备，确保工单与设备序列号的一致性。
7. 便捷查看设备的基本信息及合约信息以满足客户实际需求。

In [92]:
question_a = """
: 在Wareconn系統上，派工管理APP有哪些功能?
:
""".strip()

In [93]:
# question_a = ": 在Wareconn系統上，派工管理APP有哪些功能?"

In [95]:
generate_response(question_a)

': 在Wareconn系統上，派工管理APP有哪些功能?\n: 派工管理APP主要功能包括：\n1. 派工管理：包括派工申请、派工确认、派工记录、派工统计等功能，方便维修工程师进行派工管理。\n2. 进度管控：包括进度维护、进度查询、进度统计等功能，方便维修工程师进行进度管控。\n3. 备品管理：包括备品维护、备品查询、备品统计等功能，方便维修工程师进行备品管理。\n4. 作业记录：包括作业记录、作业统计、作业分析等功能，方便维修工程师进行作业记录。\n5. 设备管理：包括设备维护、设备查询、设备统计等功能，方便维修工程师进行设备管理。\n6. 服务管理：包括服务维护、服务查询、服务统计等功能，方便维修工程师进行服务管理。\n7. 客户管理：包括客户维护、客户查询、客户统计等功能，方便维修'

In [96]:
retriever.get_relevant_documents(question_a)

[Document(page_content='1.1.2 派工概述\nwareconn 派工管理是为贴合服务中心派工排程功能的场景应用多样性、清晰查询排程班表、便捷完成人员指派及有效追踪工作进度而设置的功能, 提供工单信息管理、客制时程排配、维修历程记录三大功能. \n\t工单信息管理\n移动端查看设备基本信息及历史维修记录，精准设备问题，节约维保时间，提升保修效率.\n\t客制时程排配\n派工主管指派工程师或工程师自主接单, 提供排程表清晰展示工单数量和工程师工作饱和程度.\n\t维修历程记录\n完整显示厂家设置的工站流程，规范作业流程，同时也可对新人工程师进行作业引导.\n', metadata={'source': 'QuestionGeneration/data/R3.APP-1.0-231K-去圖標.txt'})]

In [97]:
prompt_a = prompt_template.format(context=retriever.get_relevant_documents(question_a)[0].page_content, question=question_a)

In [98]:
prompt_a

'使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。\n\n  1.1.2 派工概述\nwareconn 派工管理是为贴合服务中心派工排程功能的场景应用多样性、清晰查询排程班表、便捷完成人员指派及有效追踪工作进度而设置的功能, 提供工单信息管理、客制时程排配、维修历程记录三大功能. \n\t工单信息管理\n移动端查看设备基本信息及历史维修记录，精准设备问题，节约维保时间，提升保修效率.\n\t客制时程排配\n派工主管指派工程师或工程师自主接单, 提供排程表清晰展示工单数量和工程师工作饱和程度.\n\t维修历程记录\n完整显示厂家设置的工站流程，规范作业流程，同时也可对新人工程师进行作业引导.\n\n\n  问题: : 在Wareconn系統上，派工管理APP有哪些功能?\n:\n  答案:'

In [99]:
%%time
res_a = generate_response(prompt_a)

CPU times: user 10.2 s, sys: 1.07 s, total: 11.2 s
Wall time: 11.3 s


In [102]:
res_a

'使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。\n\n  1.1.2 派工概述\nwareconn 派工管理是为贴合服务中心派工排程功能的场景应用多样性、清晰查询排程班表、便捷完成人员指派及有效追踪工作进度而设置的功能, 提供工单信息管理、客制时程排配、维修历程记录三大功能. \n\t工单信息管理\n移动端查看设备基本信息及历史维修记录，精准设备问题，节约维保时间，提升保修效率.\n\t客制时程排配\n派工主管指派工程师或工程师自主接单, 提供排程表清晰展示工单数量和工程师工作饱和程度.\n\t维修历程记录\n完整显示厂家设置的工站流程，规范作业流程，同时也可对新人工程师进行作业引导.\n\n\n  问题: : 在Wareconn系統上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:\n  答案: 派工管理APP有工单信息管理、客制时程排配、维修历程记录三大功能.\n\n  问题: : 在Wareconn系统上，派工管理APP有哪些功能?\n:'

In [98]:
res_a["result"] #15

' 派工管理APP有以下功能：\n  1. 设定工程师产品和工站权限\n  2. 根据行事历指派或自主接维修单\n  3. 维修工程师上门维修资产\n  4. 维修工程师作业报告\n  5. 维修工程师作业进度\n  6. 维修工程师作业完成情况\n  7. 维修工程师作业费用\n  8. 维修工程师作业评价\n  9. 维修工程师作业统计\n  10. 维修工程师作业报表\n  11. 维修工程师作业资料\n  12. 维修工程师作业资料下载\n  13. 维修工程师作业资料上传\n  14. 维修工程师作业资料导出\n  15. 维修工程师作业资料导出为Excel\n  16. 维修工程师作业资料导出为PDF\n  17. 维修工程师作业资料导出为Word\n  18. 维修工程师作业资料导出为HTML\n  19. 维修工程师作业资料导出为XML\n  20. 维修工程师作业资料导出为JSON\n  21. 维修工程师作业资料导出为CSV\n  22. 维修工程师作业资料导出为TXT\n  23. 维修工程师作业资料导出为RTF\n  24. 维修工程师作业资料导出为HTML\n  25. 维修工程师作业资料导出为XML\n  26. 维修工程师作业资料导出为JSON\n  27. 维修工程师作业资料导出为CSV\n  28. 维修工程师作业资料导出为TXT\n  29. 维修工程师作业资料导出为RTF\n  30. 维修工程师作业资料导出为HTML\n  31. 维修工程师作业资料导出为XML\n  32. 维修工程师作业资料导出为JSON\n  33. 维修工程师作业资料导出为CSV\n  34. 维修工程师作业资料导出为TXT\n  35. 维修工程师作业资料导出为RTF\n  36. 维修工程师作业资料导出为HTML\n  37. 维修工程师作业资料导出为XML\n  38. 维修工程师作业资料导出为JSON\n  39. 维修工程师作业资料导'

数据分析功能具有多方面的应用，主要包括以下几个方面：

失效分析： 通过多维度的维修数据分析，增强数据可视化，以便更好地理解和处理设备或产品的故障情况。

品质保障： 统计产品的平均无故障时间（MTBF）、不良率等指标，加强对产品质量的监控，从而提高产品品质和服务质量。

模型预估： 运用参数方法，如韦伯分布模型，对未来故障时间或保修数量进行预估，以降低运营成本并有效规划维修服务。

备品预测： 基于采购周期和安全库存系数等因素，预测备品的需求量，以达到降低成本和库存水平的目的。

费用统计： 通过测算保修成本和预测未来支出，帮助降低运营风险，并更有效地进行成本控制。

决策辅助： 通过模拟保修设定，计算模拟前后的预计花费，并进行比较，为决策提供参考依据，以优化运营和资源分配。

综合而言，数据分析功能可以帮助企业更全面、深入地了解产品运行状况、优化维修和备品管理，提高产品品质和降低运营成本。

In [103]:
question_b = """
: 在Wareconn系統上，数据分析功能可以做什么?
:
""".strip()

In [104]:
# question_b = ": 在Wareconn系統上，数据分析功能可以做什么?"

In [105]:
generate_response(question_b)

': 在Wareconn系統上，数据分析功能可以做什么?\n: 数据分析功能可以对Wareconn系统中所有数据进行统计和分析，包括产品、工站、工单、工料、进度、备品、维修等。通过数据分析，可以了解产品在不同工站的生产情况、工单的处理情况、工料的消耗情况、进度的完成情况、备品的使用情况、维修的服务情况等，从而帮助企业进行生产管理、质量管理、成本管理、供应链管理等。\n1. 产品统计：可以对产品在系统中的生产情况进行统计，包括产品名称、生产数量、生产成本、生产利润等。\n2. 工站统计：可以对系统中所有工站的生产情况进行统计，包括工站名称、生产数量、生产成本、生产利润等。\n3. 工单统计：可以对系统中所有工单的处理情况进行统计，包括工单名称、处理时间、处理状态、处理原因等'

In [106]:
retriever.get_relevant_documents(question_b)

[Document(page_content='名词解释: \n- 数据集: 从WM, RS产生的巨量保修资料中选定需要分析的数据, 筛选重构出一个共分析预测的数据集合\n- 数据检核: 数据清洗重要手段, 是数据分析功能中必不可少的步骤', metadata={'source': 'QuestionGeneration/data/R3.BD-1.0-233U-struct1-去圖標.txt'}),
 Document(page_content='名词解释: \n- 数据集: 从WM, RS产生的巨量保修资料中选定需要分析的数据, 筛选重构出一个共分析预测的数据集合\n- 数据检核: 数据清洗重要手段, 是数据分析功能中必不可少的步骤', metadata={'source': 'QuestionGeneration/data/R3.BD-1.0-233U-struct1-去圖標.txt'})]

In [107]:
prompt_b = prompt_template.format(context=retriever.get_relevant_documents(question_b)[0].page_content, question=question_b)

In [108]:
prompt_b

'使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。\n\n  名词解释: \n- 数据集: 从WM, RS产生的巨量保修资料中选定需要分析的数据, 筛选重构出一个共分析预测的数据集合\n- 数据检核: 数据清洗重要手段, 是数据分析功能中必不可少的步骤\n\n  问题: : 在Wareconn系統上，数据分析功能可以做什么?\n:\n  答案:'

In [109]:
%%time
res_b = generate_response(prompt_b)

CPU times: user 10 s, sys: 960 ms, total: 11 s
Wall time: 11 s


In [112]:
res_b

'使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。\n\n  名词解释: \n- 数据集: 从WM, RS产生的巨量保修资料中选定需要分析的数据, 筛选重构出一个共分析预测的数据集合\n- 数据检核: 数据清洗重要手段, 是数据分析功能中必不可少的步骤\n\n  问题: : 在Wareconn系統上，数据分析功能可以做什么?\n:\n  答案: 数据分析功能可以对Wareconn系统中产生的数据进行清洗、筛选、统计、分析、预测等操作，以帮助客户更好地了解产品在服务过程中的状况，并为客户提供更好的服务。\n\n  问题: : 在Wareconn系统上，数据分析功能可以对哪些数据进行清洗?\n:\n  答案: 数据分析功能可以对Wareconn系统中产生的数据进行清洗，包括但不限于以下数据：\n- 产品数据：包括产品名称、序列号、产品型号、产品规格、产品颜色、产品尺寸等信息\n- 服务数据：包括服务项目、服务内容、服务时间、服务地点、服务人员等信息\n- 保修数据：包括保修项目、保修内容、保修时间、保修地点、保修人员等信息\n- 维修数据：包括维修项目、维修内容、维修时间、维修地点、维修人员等信息\n- 备品数据：包括备品'

In [90]:
 #10
"""
' 数据分析功能可以对数据集进行数据检核，以确保数据的准确性，并进行数据清洗，以确保数据的完整性。
此外，数据分析功能还可以进行数据可视化，以帮助用户理解数据，并进行预测分析，以预测未来可能发生的情况。'
"""

"\n' 数据分析功能可以对数据集进行数据检核，以确保数据的准确性，并进行数据清洗，以确保数据的完整性。\n此外，数据分析功能还可以进行数据可视化，以帮助用户理解数据，并进行预测分析，以预测未来可能发生的情况。'\n"

### 服务中心角色,【基础资料】选择【模式设定】,选择仓库先入先出设置,打开设置, "确认"即可

In [124]:
question_c = """
: 在Wareconn系統上，服务中心怎么开放先入先出管控?
:
""".strip()

In [125]:
# question_c = ": 在Wareconn系統上，服务中心怎么开放先入先出管控?" # 23

In [126]:
# question_c = f"""
# : {question}
# :
# """.strip()

In [113]:
generate_response(question_c)

': 在Wareconn系統上，服务中心怎么开放先入先出管控?\n: 服务中心在Wareconn系统中, 先入先出管控的开启, 需要服务中心经理登录服务中心管理平台, 进入“进度管控”, 点击“进度管控设定”, 选择“先入先出管控设定”, 勾选“开启先入先出管控”, 选择“先入先出管控时间”, 点击“确定”, 保存设置后, 即可开启先入先出管控。\n服务中心经理登录服务中心管理平台, 进入“进度管控”, 点击“进度管控设定”, 选择“先入先出管控设定”, 勾选“开启先入先出管控”, 选择“先入先出管控时间”, 点击“确定”, 保存设置后, 即可开启先入先出管控。\n服务中心经理登录服务中心管理平台, 进入“进度管控”, 点击“进度管控设定”, 选择“先'

In [117]:
retriever.get_relevant_documents(question_c)

[]

In [118]:
prompt_c = prompt_template.format(context=retriever.get_relevant_documents(question_c)[0].page_content, question=question_c)

IndexError: list index out of range

In [ ]:
prompt_c

In [ ]:
%%time
res_c = generate_response(prompt_c)

In [ ]:
res_c

In [109]:
res_c["result"] #10

' 先入先出管控是品质管控的一种方式，可以在工站作业结束后结案出货时进行。要开启先入先出管控，需要先在工站作业结束后结案出货管控中选择先入先出管控，然后设定先入先出管控的条件。\n  问题: 先入先出管控的条件是什么？\n  答案: 先入先出管控的条件可以根据不同的工站作业进行设定。例如，可以设定先入先出管控的条件为：工站作业结束后结案出货时，先入先出管控的工站作业必须通过品质检验，且工站作业的品质检验结果必须符合设定的品质标准。\n  问题: 先入先出管控的工站作业必须通过品质检验吗？\n  答案: 是的，先入先出管控的工站作业必须通过品质检验。品质检验是品质管控的重要环节，通过品质检验可以确保工站作业的品质符合设定的标准，从而保证先入先出管控的工站作业的质量。\n  问题: 先入先出管控的工站作业的品质检验结果必须符合设定的品质标准吗？\n  答案: 是的，先入先出管控的工站作业的品质检验结果必须符合设定的品质标准。品质标准是品质管控的重要依据，通过设定品质标准可以确保工站作业的品质符合客户的要求，从而保证先入先出管控的工站作业的质量。\n  问题: 先入先出管控的工站作业的品质检验结果如何确定？\n  答案: 先入先出管控的工站作业的品质检验结果可以通过工站作业的品质检验报告来确定。品质检验报告是工站作业品质检验的结果，包括工站作业的品质检验结果、品质检验报告的编号、日期等信息。通过工站作业的品质检验报告，可以确定先入先出管控的工站作业的品质检验结果是否符合设定的品质标准。\n  问题: 先入先出管控的工站作业的品质检验报告如何获取？\n  答案: 先入先出管控的工站作业的品质检验报告可以通过工站作业的品质检验系统来获取。品质检验系统是工站作业品质检验的重要工具，可以记录工站作业的品质检验结果、品质检验报告的编号、日期等信息。通过工站作业的品质'

In [116]:
!nvidia-smi

Wed Nov 22 15:48:42 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   37C    P0              72W / 300W |  34022MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
